# Rastreo de Objetos

Para el campo de visión por computadora, la tarea de rastrear objetos tiene gran cantidad de aplicaciones. Hay constantes esfuerzos por parte de la academia y la industria por encontrar nuevos algoritmos que resulevan este reto de la manera mas eficiente y rápida posible, que permita, por ejemplo, el rastreo de objetos sobre videos en tiempo real.

Se procede a mostrar la aplicación de diferentes algoritmos de rastreo sobre un video, esto mediante OpenCV. Se puede seleccionar entre los siguientes algoritmos:

* BOOSTING
* KCF
* MEDIANFLOW
* MOSSE
* CSRT

Adicional a seleccionar el algoritmo, hay disponibles tres videos sobre los cuales se puede aplicar el rastreo. Las opciones son las siguientes:

* Barca.mp4
* Chaplin.mp4
* MarioKart.mp4

Todos se encuentran dentro de la carpeta "videos" (se debe agregar como parte de la ruta al video).


In [43]:
import cv2
import sys
from time import sleep

tracker_type = 'MOSSE'
video_path = 'videos/Barca.mp4'

Esta siguiente parte construye el rastreador basado en el algoritmo seleccionado por el usuario. Estas son simplemente las opciones seleccionadas como ejemplos, pero OpenCV cuanta con mas algoritmos de rastreo.

In [44]:
switcher={
    'BOOSTING':   cv2.TrackerBoosting_create(),
    'KCF':        cv2.TrackerKCF_create(),
    'MEDIANFLOW': cv2.TrackerMedianFlow_create(),
    'MOSSE':      cv2.TrackerMOSSE_create(),
    "CSRT":       cv2.TrackerCSRT_create()
}

tracker = switcher.get(tracker_type)

Posteriormente se procede a abrir el video seleccionado.

In [45]:
video = cv2.VideoCapture(video_path)
if not video.isOpened():
    print("Could not open video")

Como primer paso, se debe seleccionar dentro del video, el objeto que se desea rastrear. Esta siguiente sección obtiene el primer cuadro del video y lo muestra para poder seleccionar el objeto que se desea rastrear.

La selección del objeto se realiza mediante el mouse. Se marcará un recuadro azul sobre el objeto seleccionado y se procede a presionar ENTER o ESPACIO para finalizar la selección.

In [46]:
ok, frame = video.read()
if not ok:
    print('Cannot read video file')
    
bbox = (287, 23, 86, 320)
bbox = cv2.selectROI(frame, False)
cv2.destroyAllWindows()

Se procede a continuación a inicializar el rastreador con el primer cuadro del video y el objeto seleccionado anteriormente.

In [47]:
ok = tracker.init(frame, bbox)

Finalmente se procede a iterar sobre todos los cuadros del video, se actualiza el rastreo del objeto para cada cuadro y se procede a dibujar sobre cada cuadro la estimación de rastreo del objeto así como el tiempo que tarda en procesar cada cuadro.

In [48]:
while True:
    sleep(0.02)
    ok, frame = video.read()
    if not ok:
        cv2.destroyAllWindows()
        break

    timer = cv2.getTickCount()

    ok, bbox = tracker.update(frame)

    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

    if ok:
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

    cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
    cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

    cv2.imshow("Tracking", frame)

    k = cv2.waitKey(1) & 0xff
    if k == 27 : break